In [1]:
import pandas as pd
import pyodbc
import os
import numpy as np
import csv
import time
import datetime
import json
from urllib.request import urlopen


# ###########
# SQL Server
# ###########
source_name = 'clinibase'
sql_uid = 'CCIS_Data_Access'
sql_pwd = 'Cc1$0a!@'
server = 'QSQLA177'
driver = 'SQL Server'
database = 'Clinibase_Repository'
sql_cnxn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database}',uid=sql_uid,pwd=sql_pwd)

# #############################################################################
# funciton to validate comp
# #############################################################################

def validateJSON(jsonData):
    try:
        json.loads(jsonData)
    except ValueError as err:
        return '*****False*****'
    return 'True'

# #############################################################################
# get table list to iterate over
# #############################################################################

with open("adhoc.sql", 'r') as f:
    tablelistsql = f.read()
    tablesdf = pd.read_sql(tablelistsql, sql_cnxn) 
    tablelist = tablesdf['table_name'].tolist()

# #############################################################################
# loop through each table to output query to json
# #############################################################################
    for t in tablelist:
        print(t)
        retrieve_mu_text = open('sqlserver_to_mu_json.sql').read()
        #print(retrieve_mu_text)
        param_tbl_sql = retrieve_mu_text.replace('XXXXXXXX',t) #tablename
        param_entity_sql = param_tbl_sql.replace('YYYYYYYY',source_name) #entity name
        #print(param_entity_sql)
        mu_text = pd.read_sql(param_entity_sql, sql_cnxn)  
        mu_text['jsontext'] = mu_text['jsontext'].replace(r'\s+|\\n', ' ', regex=True) 

        with open(os.path.join(os.getcwd(),'json',f'{source_name}_{t}.json'),'a') as write_file:
            mutexttojson = mu_text.to_string(header=False, index=False)
            #print(mutexttojson)
            json_out = json.dumps(json.loads(mutexttojson),indent=1)
            open(os.path.join(os.getcwd(),'json',f'{source_name}_{t}.json'),'w').write(json_out)
            

ClinibaseImpactDiagnosis
ClinibaseSensisProcedurePerformed
procedures1


In [19]:

# #############################################################################
# loop through new json file to validate
# #############################################################################
directory = os.path.join(os.getcwd(),'json')
for jsonfiles in os.listdir(directory):
     jsonfilename = os.fsdecode(jsonfiles)
     if jsonfilename.endswith(".json"):
        with open(os.path.join(directory, jsonfilename), 'r') as j:
            isValid = validateJSON(j.read())
            print(f'{jsonfiles} is Valid', isValid)
     else:
         continue            